In [1]:
import numpy as np
import tensorflow as tf
import time

2024-04-28 11:53:06.467845: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-28 11:53:06.521738: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-28 11:53:06.521775: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-28 11:53:06.523295: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-28 11:53:06.532589: I tensorflow/core/platform/cpu_feature_guar

In [7]:
def convert_to_tflite(model, input_size, n):
    def representative_dataset():
        for _ in range(100):
            yield [tf.random.normal([1, *input_size])]

    quantizer = tf.lite.TFLiteConverter.from_keras_model(model)
    quantizer.optimizations = [tf.lite.Optimize.DEFAULT]
    quantizer.representative_dataset = representative_dataset
    quantizer.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]
    quantizer.inference_input_type = tf.uint8  # or tf.uint8
    quantizer.inference_output_type = tf.uint8  # or tf.uint8
    
    tflite_quant_model = quantizer.convert()

    # Save the TFLite model to a file
    with open(f"models/conv2/TPU_CNN2_{n}.tflite", "wb") as f:
        f.write(tflite_quant_model)
    print("saved tf lite model")
    return tflite_quant_model

In [5]:
# figure out intermediete output sizes
from architectures import conv2
model = conv2()
model.build((None, 128, 128, 3))
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 126, 126, 32)      896       
                                                                 
 depthwise_conv2d_6 (Depthw  (None, 124, 124, 32)      320       
 iseConv2D)                                                      
                                                                 
 conv2d_13 (Conv2D)          (None, 122, 122, 64)      18496     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 61, 61, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 59, 59, 64)        36928     
                                                                 
 depthwise_conv2d_7 (Depthw  (None, 57, 57, 64)       

In [9]:
src = [
    [
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.DepthwiseConv2D((3, 3), activation='relu'),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2))
    ],  
    [
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.DepthwiseConv2D((3, 3), activation='relu'),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.GlobalAveragePooling2D()
    ],  
    [
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.2)
    ],
    [
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ]
]
input_shapes = [(None,128,128,3), (None, 61, 61, 64), (None,128, ), (None,512,)]
for i in range(0,4):
    a= []
    b = []
    for layer_group in src[i:]:
        a.extend(layer_group)
    if a:
        a = tf.keras.Sequential(a)
        a.build(input_shapes[i])
        a.save(f"models/conv2/GPU_CNN2_{i+4}.keras")
        convert_to_tflite(a, input_shapes[i][1:], i)
    
    for layer_group in src[:i]:
        b.extend(layer_group)
    if b:
        b = tf.keras.Sequential(b)
        b.build((None, 128, 128, 3))
        b.save(f"models/conv2/GPU_CNN2_{i}.keras")
        convert_to_tflite(b, (128,128,3), i+4)


INFO:tensorflow:Assets written to: /tmp/tmpugwfogxl/assets


INFO:tensorflow:Assets written to: /tmp/tmpugwfogxl/assets
/u/pj8wfq/rl/MC/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-04-28 11:58:30.013599: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-04-28 11:58:30.013633: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-04-28 11:58:30.013845: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpugwfogxl
2024-04-28 11:58:30.016287: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-04-28 11:58:30.016308: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpugwfogxl
2024-04-28 11:58:30.021818: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-04-28 11:58:30.067741:

saved tf lite model
INFO:tensorflow:Assets written to: /tmp/tmp4_67rcls/assets


INFO:tensorflow:Assets written to: /tmp/tmp4_67rcls/assets
/u/pj8wfq/rl/MC/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-04-28 11:58:37.036722: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-04-28 11:58:37.036758: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-04-28 11:58:37.037022: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp4_67rcls
2024-04-28 11:58:37.038764: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-04-28 11:58:37.038779: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp4_67rcls
2024-04-28 11:58:37.042940: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-04-28 11:58:37.078718:

saved tf lite model
INFO:tensorflow:Assets written to: /tmp/tmp7_5p0a9u/assets


...

INFO:tensorflow:Assets written to: /tmp/tmpfshcinqs/assets
/u/pj8wfq/rl/MC/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-04-28 11:58:52.008529: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-04-28 11:58:52.008561: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-04-28 11:58:52.008772: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpfshcinqs
2024-04-28 11:58:52.009412: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-04-28 11:58:52.009425: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpfshcinqs
2024-04-28 11:58:52.011194: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-04-28 11:58:52.027268:

saved tf lite model
INFO:tensorflow:Assets written to: /tmp/tmpaixv_jje/assets


INFO:tensorflow:Assets written to: /tmp/tmpaixv_jje/assets
/u/pj8wfq/rl/MC/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-04-28 11:58:53.521248: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-04-28 11:58:53.521278: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-04-28 11:58:53.521471: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpaixv_jje
2024-04-28 11:58:53.524304: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-04-28 11:58:53.524324: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpaixv_jje
2024-04-28 11:58:53.529710: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-04-28 11:58:53.570826:

saved tf lite model


fully_quantize: 0, inference_type: 6, input_inference_type: UINT8, output_inference_type: UINT8
